In [1]:
%matplotlib inline

# https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

In [3]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
from functools import reduce
import os
import pickle

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import sys
sys.path.insert(0, "D:\\Documents\\food_recipe_gen\\recipe_1m_analysis")

from utils import Vocabulary, FOLDER_PATH, DATA_FILES 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Similar to the character encoding used in the character-level RNN
tutorials, we will be representing each word in a language as a one-hot
vector, or giant vector of zeros except for a single one (at the index
of the word). Compared to the dozens of characters that might exist in a
language, there are many many more words, so the encoding vector is much
larger. We will however cheat a bit and trim the data to only use a few
thousand words per language.

.. figure:: /_static/img/seq-seq-images/word-encoding.png
   :alt:





We'll need a unique index per word to use as the inputs and targets of
the networks later. To keep track of all this we will use a helper class
called ``Lang`` which has word → index (``word2index``) and index → word
(``index2word``) dictionaries, as well as a count of each word
``word2count`` to use to later replace rare words.




In [4]:
SOS_token = 0
EOS_token = 1

The full process for preparing the data is:

-  Read text file and split into lines, split lines into pairs
-  Normalize text, filter by length and content
-  Make word lists from sentences in pairs




In [5]:
DATA_FILES

['allingrs_count.pkl',
 'allwords_count.pkl',
 'recipe1m_test.pkl',
 'recipe1m_vocab_ingrs.pkl',
 'recipe1m_vocab_toks.pkl']

In [6]:
with open(os.path.join(FOLDER_PATH,DATA_FILES[3]),'rb') as f:
    vocab_ingrs=pickle.load(f)
    
with open(os.path.join(FOLDER_PATH,DATA_FILES[4]),'rb') as f:
    vocab_tokens=pickle.load(f)

In [7]:
with open(os.path.join(FOLDER_PATH,DATA_FILES[2]),'rb') as f:
    data=pickle.load(f)

pairs=[]
for recipe in data:
    pairs.append([recipe["ingredients"],recipe["tokenized"]])

print(len(pairs))

18745


In [8]:
length=0
for i,pair in enumerate(pairs):
#     length_t=reduce(lambda x,y:len(x)+len(y), pair[1])
    length_t=0
    for sent in pair[1]:
        length_t+=len(sent)
    
    if length_t>length:
        length=length_t

length

539

In [9]:
MAX_LENGTH=300

In [10]:
# pairs[16826]

In [11]:
def filterPair(p):
    length=0
    for ingr in p[0]:
        if ingr not in vocab_ingrs.word2idx:
            return False
        
    for sent in p[1]:
        
        for word in sent:
            # TODO check how steps tokenized ? Put into vocab ???
            if word not in vocab_tokens.word2idx:
                return False
        length+=len(sent)
    
    return length < MAX_LENGTH


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

pairs = filterPairs(pairs)
print(len(pairs))

14724


In [13]:
# vocab_ingrs.word2idx['quahogs']

<div class="alert alert-info"><h4>Note</h4><p>There are other forms of attention that work around the length
  limitation by using a relative position approach. Read about "local
  attention" in `Effective Approaches to Attention-based Neural Machine
  Translation <https://arxiv.org/abs/1508.04025>`__.</p></div>

Training
========

Preparing Training Data
-----------------------

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.




In [14]:
def list2idx(vocab, sentence):
    return [vocab.word2idx[word] for word in sentence]


def tensorFromSentence(vocab, sentence,instructions=False):
    if instructions:
        indexes=[]
        for sent in sentence:
            indexes.extend(list2idx(vocab, sent))
    else:
        indexes = list2idx(vocab, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(vocab_ingrs, pair[0])
    target_tensor = tensorFromSentence(vocab_tokens, pair[1],instructions=True)
    return (input_tensor, target_tensor)

# for i,pair in enumerate(pairs):
#     try:
#         tensorsFromPair(pair)
#     except KeyError as e:
#         print(e)
#         print(i)
#         print(pair)
#         break

The Seq2Seq Model
=================

A Recurrent Neural Network, or RNN, is a network that operates on a
sequence and uses its own output as input for subsequent steps.

A `Sequence to Sequence network <https://arxiv.org/abs/1409.3215>`__, or
seq2seq network, or `Encoder Decoder
network <https://arxiv.org/pdf/1406.1078v3.pdf>`__, is a model
consisting of two RNNs called the encoder and decoder. The encoder reads
an input sequence and outputs a single vector, and the decoder reads
that vector to produce an output sequence.

.. figure:: /_static/img/seq-seq-images/seq2seq.png
   :alt:

Unlike sequence prediction with a single RNN, where every input
corresponds to an output, the seq2seq model frees us from sequence
length and order, which makes it ideal for translation between two
languages.

Consider the sentence "Je ne suis pas le chat noir" → "I am not the
black cat". Most of the words in the input sentence have a direct
translation in the output sentence, but are in slightly different
orders, e.g. "chat noir" and "black cat". Because of the "ne/pas"
construction there is also one more word in the input sentence. It would
be difficult to produce a correct translation directly from the sequence
of input words.

With a seq2seq model the encoder creates a single vector which, in the
ideal case, encodes the "meaning" of the input sequence into a single
vector — a single point in some N dimensional space of sentences.




The Encoder
-----------

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.



In [15]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder
-----------

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to create the translation.




**Simple Decoder**


In the simplest seq2seq decoder we use only last output of the encoder.
This last output is sometimes called the *context vector* as it encodes
context from the entire sequence. This context vector is used as the
initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and
hidden state. The initial input token is the start-of-string ``<SOS>``
token, and the first hidden state is the context vector (the encoder's
last hidden state).




In [16]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

I encourage you to train and observe the results of this model, but to
save space we'll be going straight for the gold and introducing the
Attention Mechanism.




**Attention Decoder**

If only the context vector is passed betweeen the encoder and decoder,
that single vector carries the burden of encoding the entire sentence.

Attention allows the decoder network to "focus" on a different part of
the encoder's outputs for every step of the decoder's own outputs. First
we calculate a set of *attention weights*. These will be multiplied by
the encoder output vectors to create a weighted combination. The result
(called ``attn_applied`` in the code) should contain information about
that specific part of the input sequence, and thus help the decoder
choose the right output words.

.. figure:: https://i.imgur.com/1152PYf.png
   :alt:

Calculating the attention weights is done with another feed-forward
layer ``attn``, using the decoder's input and hidden state as inputs.
Because there are sentences of all sizes in the training data, to
actually create and train this layer we have to choose a maximum
sentence length (input length, for encoder outputs) that it can apply
to. Sentences of the maximum length will use all the attention weights,
while shorter sentences will only use the first few.

.. figure:: /_static/img/seq-seq-images/attention-decoder-network.png
   :alt:





In [17]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

Training the Model
------------------

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the ``<SOS>`` token as its first input, and the last hidden state of the
encoder as its first hidden state.

"Teacher forcing" is the concept of using the real target outputs as
each next input, instead of using the decoder's guess as the next input.
Using teacher forcing causes it to converge faster but `when the trained
network is exploited, it may exhibit
instability <http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.378.4095&rep=rep1&type=pdf>`__.

You can observe outputs of teacher-forced networks that read with
coherent grammar but wander far from the correct translation -
intuitively it has learned to represent the output grammar and can "pick
up" the meaning once the teacher tells it the first few words, but it
has not properly learned how to create the sentence from the translation
in the first place.

Because of the freedom PyTorch's autograd gives us, we can randomly
choose to use teacher forcing or not with a simple if statement. Turn
``teacher_forcing_ratio`` up to use more of it.




In [35]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH, attention=False):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        if attention:
            for di in range(target_length):
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
                loss += criterion(decoder_output, target_tensor[di])
                decoder_input = target_tensor[di]  # Teacher forcing
        else:
            for di in range(target_length):
                decoder_output, decoder_hidden = decoder(
                    decoder_input, decoder_hidden)
                loss += criterion(decoder_output, target_tensor[di])
                decoder_input = target_tensor[di]  # Teacher forcing
            
    else:
        # Without teacher forcing: use its own predictions as the next input
        if attention:
            for di in range(target_length):
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze().detach()  # detach from history as input

                loss += criterion(decoder_output, target_tensor[di])
                if decoder_input.item() == EOS_token:
                    break
        else:
            for di in range(target_length):
                decoder_output, decoder_hidden = decoder(
                    decoder_input, decoder_hidden)
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze().detach()  # detach from history as input

                loss += criterion(decoder_output, target_tensor[di])
                if decoder_input.item() == EOS_token:
                    break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.




In [36]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.




In [37]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01,attention=True):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion,attention=attention)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

Plotting results
----------------

Plotting is done with matplotlib, using the array of loss values
``plot_losses`` saved while training.




In [21]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

Evaluation
==========

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder's
attention outputs for display later.




In [30]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(vocab_ingrs, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(vocab_tokens.idx2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [28]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

Training and Evaluating
=======================

With all these helper functions in place (it looks like extra work, but
it makes it easier to run multiple experiments) we can actually
initialize a network and start training.

Remember that the input sentences were heavily filtered. For this small
dataset we can use relatively small networks of 256 hidden nodes and a
single GRU layer. After about 40 minutes on a MacBook CPU we'll get some
reasonable results.

.. Note::
   If you run this notebook you can train, interrupt the kernel,
   evaluate, and continue training later. Comment out the lines where the
   encoder and decoder are initialized and run ``trainIters`` again.




In [24]:
hidden_size = 256
encoder1 = EncoderRNN(len(vocab_ingrs.idx2word.keys()), hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, len(vocab_tokens.idx2word.keys()), dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 5000, print_every=100)

1m 27s (- 144m 29s) (100 1%) 20.5818
2m 49s (- 138m 23s) (200 2%) 25.0482
4m 13s (- 136m 36s) (300 3%) 32.8347
5m 37s (- 135m 5s) (400 4%) 36.4157
6m 56s (- 131m 54s) (500 5%) 37.1037
8m 22s (- 131m 12s) (600 6%) 42.1024
9m 47s (- 130m 10s) (700 7%) 43.1715
11m 8s (- 128m 12s) (800 8%) 48.5861
12m 31s (- 126m 39s) (900 9%) 49.4105
14m 3s (- 126m 31s) (1000 10%) 50.4300
15m 37s (- 126m 21s) (1100 11%) 51.5683
16m 57s (- 124m 24s) (1200 12%) 45.9525
18m 37s (- 124m 40s) (1300 13%) 60.4985
20m 0s (- 122m 52s) (1400 14%) 51.0973
21m 36s (- 122m 27s) (1500 15%) 53.4614
23m 2s (- 120m 58s) (1600 16%) 50.1443
24m 31s (- 119m 42s) (1700 17%) 52.3147
25m 58s (- 118m 18s) (1800 18%) 56.0034
27m 20s (- 116m 34s) (1900 19%) 49.9412
28m 37s (- 114m 31s) (2000 20%) 49.9150
30m 4s (- 113m 9s) (2100 21%) 53.6711
31m 25s (- 111m 26s) (2200 22%) 49.7835
32m 53s (- 110m 6s) (2300 23%) 57.0558
34m 18s (- 108m 38s) (2400 24%) 56.2472
35m 42s (- 107m 7s) (2500 25%) 55.0453
37m 5s (- 105m 35s) (2600 26%) 60.

In [ ]:
hidden_size = 256
encoder2 = EncoderRNN(len(vocab_ingrs.idx2word.keys()), hidden_size).to(device)
attn_decoder2 = DecoderRNN(hidden_size, len(vocab_tokens.idx2word.keys())).to(device)

trainIters(encoder2, attn_decoder2, 5000, print_every=100,attention=False)

1m 7s (- 54m 51s) (100 2%) 35.4625
2m 14s (- 53m 42s) (200 4%) 47.4241
3m 27s (- 54m 18s) (300 6%) 56.7911
4m 41s (- 53m 51s) (400 8%) 58.8278
5m 52s (- 52m 54s) (500 10%) 65.7322
7m 6s (- 52m 6s) (600 12%) 68.7951
8m 22s (- 51m 24s) (700 14%) 76.7185
9m 35s (- 50m 19s) (800 16%) 70.3571
10m 43s (- 48m 52s) (900 18%) 65.3358
11m 51s (- 47m 25s) (1000 20%) 71.5617
13m 1s (- 46m 9s) (1100 22%) 64.5580
14m 12s (- 44m 58s) (1200 24%) 66.0643
15m 21s (- 43m 41s) (1300 26%) 67.3903
16m 29s (- 42m 24s) (1400 28%) 66.9448
17m 37s (- 41m 7s) (1500 30%) 67.3278
18m 52s (- 40m 6s) (1600 32%) 70.7842
20m 7s (- 39m 4s) (1700 34%) 77.2261
21m 12s (- 37m 42s) (1800 36%) 67.4372
22m 21s (- 36m 29s) (1900 38%) 67.6995
23m 28s (- 35m 12s) (2000 40%) 74.0313
24m 36s (- 33m 59s) (2100 42%) 75.3026
25m 48s (- 32m 50s) (2200 44%) 82.8352
26m 55s (- 31m 36s) (2300 46%) 75.1532
28m 10s (- 30m 31s) (2400 48%) 83.6360
29m 21s (- 29m 21s) (2500 50%) 86.0991
30m 34s (- 28m 13s) (2600 52%) 85.6719
31m 44s (- 27m 2

In [31]:
evaluateRandomly(encoder1, attn_decoder1)

> ['cream_cheese', 'sour_cream', 'ground_cumin', 'lime_zest', 'pepper', 'barbecue_sauce', 'tomatoes', 'yellow_peppers', 'green_onions', 'cheddar_cheese', 'tortilla_chips']
= [['beat', 'cream', 'cheese', 'with', 'the', 'sour', 'cream', ',', 'cumin', ',', 'lime', 'zest', '.'], ['pepper', 'and', 'half', 'of', 'the', 'diana', 'sauce', 'until', 'smooth', '.'], ['spread', 'into', 'an', '11x7-inch', 'serving', 'dish', '.'], ['toss', 'the', 'remaining', 'diana', 'sauce', 'with', 'tomato', ',', 'yellow', 'pepper', 'and', 'half', 'the', 'green', 'onion', ';', 'spoon', 'evenly', 'over', 'the', 'cream', 'cheese', 'layer', '.'], ['sprinkle', 'with', 'cheddar', 'cheese', 'and', 'the', 'remaining', 'green', 'onions', '.'], ['serve', 'with', 'tortilla', 'chips', '.'], ['tip', ':', 'make', 'your', 'own', 'grilled', 'tortilla', 'chips', 'by', 'cutting', 'flour', 'tortillas', 'into', 'wedges', 'and', 'tossing', 'with', 'a', 'little', 'vegetable', 'oil', '.'], ['grill', 'over', 'medium-low', 'heat', 'unti

< preheat honey honey honey honey honey juice and honey and honey juice water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes water minutes wate

= [['spread', 'croutons', 'in', 'a', 'lightly', 'greased', '13', 'x', '9', 'x', '2', 'inch', 'pan', '.'], ['set', 'aside', '.'], ['cook', 'sausage', ',', 'drain', '.'], ['crumble', '.'], ['sprinkle', 'sausage', 'over', 'croutons', '.'], ['combine', 'egg', ',', 'lowfat', 'milk', ',', 'soup', ',', 'mushrooms', ',', 'and', 'dry', 'mustard', '-', 'mix', 'well', 'and', 'pour', 'over', 'sausage', '.'], ['cover', 'and', 'chill', 'overnight', 'or', 'possibly', 'at', 'least', '8', 'hrs', '.'], ['remove', 'from', 'refrigerator', '-', 'let', 'stand', '30', 'min', '.'], ['bake', 'uncovered', 'at', '325', 'degrees', 'for', '50', 'to', '55', 'min', '.'], ['sprinkle', 'cheese', 'over', 'top', 'and', 'bake', 'an', 'additional', '5', 'min', 'or', 'possibly', 'till', 'cheese', 'melts', '.']]
< and . grill and salt and and and and and honey with salt honey salt honey salt honey salt honey salt honey honey salt honey salt honey salt honey salt honey salt honey salt honey salt honey salt honey salt . honey

< and salt . get mixture cheese thoroughly thoroughly get cheese get cheese turkey cheese get cheese thoroughly thoroughly thoroughly cheese get cheese get cheese get cheese get cheese get cheese get cheese get cheese get cheese thoroughly cheese get cheese get cheese get cheese get cheese get cheese get cheese get cheese get cheese get cheese cheese get cheese get cheese get cheese get cheese get cheese turkey cheese turkey cheese get cheese get cheese get cheese get cheese get cheese get cheese get cheese get cheese thoroughly cheese get cheese get cheese get cheese get cheese get cheese get cheese get cheese get cheese get cheese get cheese turkey cheese get cheese turkey cheese get cheese turkey cheese get cheese get cheese get cheese get cheese thoroughly cheese get cheese thoroughly thoroughly cheese get cheese get cheese get cheese get cheese get cheese get cheese turkey cheese get cheese get cheese get cheese get cheese get cheese get cheese get cheese get cheese get cheese tur

Visualizing Attention
---------------------

A useful property of the attention mechanism is its highly interpretable
outputs. Because it is used to weight specific encoder outputs of the
input sequence, we can imagine looking where the network is focused most
at each time step.

You could simply run ``plt.matshow(attentions)`` to see attention output
displayed as a matrix, with the columns being input steps and rows being
output steps:




In [ ]:
output_words, attentions = evaluate(
    encoder1, attn_decoder1, "je suis trop froid .")
plt.matshow(attentions.numpy())

For a better viewing experience we will do the extra work of adding axes
and labels:




In [ ]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)


evaluateAndShowAttention("elle a cinq ans de moins que moi .")

evaluateAndShowAttention("elle est trop petit .")

evaluateAndShowAttention("je ne crains pas de mourir .")

evaluateAndShowAttention("c est un jeune directeur plein de talent .")

Exercises
=========

-  Replace the embeddings with pre-trained word embeddings such as word2vec or
   GloVe
-  Try with more layers, more hidden units, and more sentences. Compare
   the training time and results.
-  If you use a translation file where pairs have two of the same phrase
   (``I am test \t I am test``), you can use this as an autoencoder. Try
   this:

   -  Train as an autoencoder
   -  Save only the Encoder network
   -  Train a new Decoder for translation from there


